<a href="https://colab.research.google.com/github/Vaibhavyadav350/Movie-Recommendation-System/blob/main/Movie_Recommendation_System_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Importing the basic libraries
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import random
from sklearn.metrics import mean_squared_error
import os
import matplotlib.pyplot as plt
from scipy.sparse.linalg import svds
from mpl_toolkits import mplot3d

In [ ]:
ratingData = pd.io.parsers.read_csv('ratings.dat', 
    names=['user_id', 'movie_id', 'rating', 'time'],
    engine='python', delimiter='::')
movieData = pd.io.parsers.read_csv('movies.dat',
    names=['movie_id', 'title', 'genre'],
    engine='python', delimiter='::', encoding='ISO-8859-1')
print(movieData)

      movie_id                               title  \
0            1                    Toy Story (1995)   
1            2                      Jumanji (1995)   
2            3             Grumpier Old Men (1995)   
3            4            Waiting to Exhale (1995)   
4            5  Father of the Bride Part II (1995)   
...        ...                                 ...   
3878      3948             Meet the Parents (2000)   
3879      3949          Requiem for a Dream (2000)   
3880      3950                    Tigerland (2000)   
3881      3951             Two Family House (2000)   
3882      3952               Contender, The (2000)   

                             genre  
0      Animation|Children's|Comedy  
1     Adventure|Children's|Fantasy  
2                   Comedy|Romance  
3                     Comedy|Drama  
4                           Comedy  
...                            ...  
3878                        Comedy  
3879                         Drama  
3880              

In [ ]:
ratingMatrix = np.ndarray(
    shape=(np.max(ratingData.movie_id.values), np.max(ratingData.user_id.values)),
    dtype=np.uint8)
ratingMatrix[ratingData.movie_id.values-1, ratingData.user_id.values-1] = ratingData.rating.values
print(ratingMatrix)

[[  5 224 143 ...   0 112   5]
 [144  41  38 ... 234  81  88]
 [ 38 127   0 ...  41  38 127]
 ...
 [  0   0   0 ...   3   0   0]
 [  0   0   0 ...   4   0   0]
 [  0   0  90 ...   4   0   0]]


In [ ]:
normalizedMatrix = ratingMatrix - np.asarray([(np.mean(ratingMatrix, 1))]).T
print(normalizedMatrix)

[[-53.41025641 165.58974359  84.58974359 ... -58.41025641  53.58974359
  -53.41025641]
 [ 78.48717949 -24.51282051 -27.51282051 ... 168.48717949  15.48717949
   22.48717949]
 [-28.84382284  60.15617716 -66.84382284 ... -25.84382284 -28.84382284
   60.15617716]
 ...
 [-10.98368298 -10.98368298 -10.98368298 ...  -7.98368298 -10.98368298
  -10.98368298]
 [-11.26223776 -11.26223776 -11.26223776 ...  -7.26223776 -11.26223776
  -11.26223776]
 [ -4.502331    -4.502331    85.497669   ...  -0.502331    -4.502331
   -4.502331  ]]


In [ ]:
A = normalizedMatrix.T / np.sqrt(ratingMatrix.shape[0] - 1)
U, S, V = np.linalg.svd(A)

In [ ]:
def similar(ratingData, movie_id, top_n):
    index = movie_id - 1 # Movie id starts from 1
    movie_row = ratingData[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', ratingData, ratingData)) #Einstein summation |  traditional matrix multiplication and is equivalent to np.matmul(a,b)
    similarity = np.dot(movie_row, ratingData.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity) #Perform an indirect sort along the given axis (Last axis)
    return sort_indexes[:top_n]

In [ ]:
k = 50
movie_id = int(input("Enter id of Movie = "))
top_n = int(input("How many recommendtions you want = "))

sliced = V.T[:, :k] # representative data
indexes = similar(sliced, movie_id, top_n)

print('Recommendations for Movie {0}: \n'.format(
movieData[movieData.movie_id == movie_id].title.values[0]))
for id in indexes + 1:
    print(movieData[movieData.movie_id == id].title.values[0])

Enter id of Movie = 34
How many recommendtions you want = 5
Recommendations for Movie Babe (1995): 

Babe (1995)
Margaret's Museum (1995)
Four Rooms (1995)
Raising Arizona (1987)
Man of the House (1995)
